In [2]:
import pinecone

with open('../secrets', 'r') as fp:
    API_KEY = fp.read()  # get api key app.pinecone.io

pinecone.init(
    api_key=API_KEY,
    environment='us-west1-gcp'
)

index = pinecone.Index('nocd-search')

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-tas-b', device='cpu')
model.max_seq_length = 256

model


/home/rezamohideen/myProjects/semantic-search/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [5]:
def convert_url(passage):
    sep = passage.split(',')
    start = sep[0].split(' ')[:4]
    end = sep[-1].split(' ')[-4:]

    start = '%20'.join(start)
    end = '%20'.join(end)

    return start + ',' + end

def convert_url_v2(passage):
    sep = passage.split(',')
    start = sep[0].split(' ')[:4]
    end = sep[-1].split(' ')[-4:]
    if len(start) < 2:
        end = sep[-1].split(' ')[-6:]

    start = '%20'.join(start)
    end = '%20'.join(end)

    return start + ',' + end

def query_db(query, model, index):
    query_emb = model.encode(query, convert_to_tensor=True, show_progress_bar=False)
    res = index.query(query_emb.tolist(), top_k=10, include_metadata=True)

    nocd = 'https://www.treatmyocd.com'

    print(f'Search Query: {query}\n')
    print('---------------------------------------------------------------------------------------------------------------------')
    print('Results\n')

    
    for item in res.matches:
        print(f"Article: {nocd}{item['metadata']['article_name']}#:~:text={convert_url(item['metadata']['text'])}")
        print(f"Paragraph Header: {item['metadata']['paragraph_name']}")
        print(f"{item['score']} {item['metadata']['text'][:150]}...\n")

In [7]:
from IPython.display import clear_output

while True:
    clear_output(wait=True)
    query = input("Search NOCD: ")
    if query == 'quit': break
    query_db(query=query, model=model, index=index)

Search Query: pocd

---------------------------------------------------------------------------------------------------------------------
Results

Article: https://www.treatmyocd.com/blog/how-long-does-pocd-last-pedophilia-ocd-symptoms-treatment#:~:text=What%20is%20POCD%20and,how%20is%20it%20treated?
Paragraph Header: What is POCD and how is it treated?
110.135193 What is POCD and how is it treated?...

Article: https://www.treatmyocd.com/blog/erp-is-effective-for-pocd#:~:text=For%20individuals%20with%20POCD,this%20might%20look%20like: 
Paragraph Header: 
108.642166 For individuals with POCD, this might look like: ...

Article: https://www.treatmyocd.com/blog/pocd-arousal-not-sign-real-desire#:~:text=Why%20POCD%20Arousal%20Is,Sign%20of%20Real%20Desire
Paragraph Header: title
102.735527 Why POCD Arousal Is Not a Sign of Real Desire...

Article: https://www.treatmyocd.com/blog/erp-is-effective-for-pocd#:~:text=These%20are%20just%20some,repeat%20in%20their%20mind. 
Paragraph Header: 
101.